In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
# import load model and save model
from tensorflow.keras.models import load_model, save_model, model_from_json, model_from_yaml
import os
import random
import time

In [ ]:
# load the pictures from clothes folder
clothes_folder = os.path.join("./clothes/")
# list the files in the folder
clothes_files = os.listdir(clothes_folder)
# print the list of files
print(clothes_files)

# load the pictures from clothes folder
modelsfolder = os.path.join("./models")
# list the files in the folder
modelsfiles = os.listdir(modelsfolder)
# print the list of files
#print(modelsfiles)


# load the pictures from clothes folder
trashfolder = os.path.join("./TrashPics")
# list the files in the folder
trashfiles = os.listdir(trashfolder)
# print the list of files
#print(trashfiles)

In [ ]:
# now store the images in a list, their paths in a list and assign them label 0
images = []
images_path = []
labels = []
for file in clothes_files:
    images_path.append(os.path.join(clothes_folder, file))
    images.append(tf.keras.preprocessing.image.load_img(os.path.join(clothes_folder, file), target_size=(227, 227)))
    labels.append(0)


for file in modelsfiles:
    images_path.append(os.path.join(modelsfolder, file))
    images.append(tf.keras.preprocessing.image.load_img(os.path.join(modelsfolder, file), target_size=(227, 227)))
    labels.append(1)
    
    
for file in trashfiles:
    images_path.append(os.path.join(trashfolder, file))
    images.append(tf.keras.preprocessing.image.load_img(os.path.join(trashfolder, file), target_size=(227, 227)))
    labels.append(2)

In [ ]:
# create a csv using these lists 
import pandas as pd 
data = {'Images': images, 'Labels': labels, 'Images Path': images_path}
df = pd.DataFrame(data)
df.to_csv('images.csv', index=False)
# now load the csv
df = pd.read_csv('images.csv')
# print the dataframe
df.head()

In [ ]:
df.info()

In [ ]:
# shuffle the dataset 
df = df.sample(frac=1).reset_index(drop=True)
# print the dataframe
df.head()

In [ ]:
# now we will load the model best.keras 
model = load_model("./best.keras")
model.summary()

In [ ]:
# now we will train this model again on our dataset
# first we will split the dataset into train and test
from sklearn.model_selection import train_test_split
# given the image, we have to predict the label
X = df['Images Path']
y = df['Labels']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# print the shape of the train and test sets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# now we will create a data generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create a data generator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
# create a train generator
train_generator = datagen.flow_from_dataframe(df, x_col='Images Path', y_col='Labels', subset='training', target_size=(227, 227), class_mode='raw')
# create a validation generator
val_generator = datagen.flow_from_dataframe(df, x_col='Images Path', y_col='Labels', subset='validation', target_size=(227, 227), class_mode='raw')
# now we will train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, validation_data=val_generator, epochs=10)


In [ ]:
# plot the training and validation accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
# take a picture and test it 
pic = cv2.imread('hello.jpg')
pic = cv2.cvtColor(pic, cv2.COLOR_BGR2RGB)
pic = cv2.resize(pic, (227, 227))
pic = np.array(pic)
pic = np.expand_dims(pic, axis=0)
#print(pic)

# now we will predict the class of the image
prediction = model.predict(pic)
print(prediction)
# get the class with the highest probability
print(np.argmax(prediction))

In [43]:
# save the model and history 
model.save('finetuned.keras')
import pickle
with open('history.pkl', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)